In [2]:
import pandas as pd

In [2]:
%%time
df = pd.read_table('sorted_output/perl_PCAWG.sorted.tsv', header=None)

CPU times: user 1.09 s, sys: 179 ms, total: 1.27 s
Wall time: 1.3 s


In [3]:
def read_junctions(filename):
    junctions = {}

    with open(filename, 'r') as junctions_file:
        for line in junctions_file:
            junction_id, offset, *counts = line.strip().split('\t')
            junctions[(junction_id, int(offset))] = list(map(int, counts))
    return junctions

In [4]:
%%time
j = read_junctions('sorted_output/perl_PCAWG.sorted.tsv')

CPU times: user 13 s, sys: 461 ms, total: 13.4 s
Wall time: 13.4 s


In [5]:
df.head()

,0,1,2,3,4,5
0,10_100008748_100010822,1,1,0,0,0
1,10_100008748_100010822,2,1,0,0,0
2,10_100008748_100010822,5,1,0,0,0
3,10_100008748_100010822,7,2,0,0,1
4,10_100008748_100010822,8,0,0,1,0


In [6]:
df.columns = ['junction_id', 'offset', 'F1', 'R1', 'F2', 'R2']

In [7]:
df = df.set_index(['junction_id', 'offset'])
df.head()

F1  R1  F2  R2
junction_id            offset                
10_100008748_100010822 1        1   0   0   0
                       2        1   0   0   0
                       5        1   0   0   0
                       7        2   0   0   1
                       8        0   0   1   0

In [8]:
%%time
df['F1+R2'] = df['F1'] + df['R2']
df['F2+R1'] = df['F2'] + df['R1']

CPU times: user 19.5 ms, sys: 12.2 ms, total: 31.7 ms
Wall time: 38.5 ms


In [9]:
df.head()

F1  R1  F2  R2  F1+R2  F2+R1
junction_id            offset                              
10_100008748_100010822 1        1   0   0   0      1      0
                       2        1   0   0   0      1      0
                       5        1   0   0   0      1      0
                       7        2   0   0   1      3      0
                       8        0   0   1   0      0      1

In [10]:
cormat = df.corr()
cormat

,F1,R1,F2,R2,F1+R2,F2+R1
F1,1.000000,0.276419,0.995020,0.276458,0.791401,0.797865
R1,0.276419,1.000000,0.266400,0.993856,0.802330,0.796798
F2,0.995020,0.266400,1.000000,0.266281,0.781862,0.794677
R2,0.276458,0.993856,0.266281,1.000000,0.806262,0.792854
F1+R2,0.791401,0.802330,0.781862,0.806262,1.000000,0.995451
F2+R1,0.797865,0.796798,0.794677,0.792854,0.995451,1.000000


In [11]:
cormat.loc['F1+R2', 'F2+R1']

0.9954506898389341

In [12]:
df.groupby(by='offset').sum()

,F1,R1,F2,R2,F1+R2,F2+R1
offset,,,,,,
1,1013947,259748,1134687,248165,1262112,1394435
2,234230,190540,218525,174998,409228,409065
3,316878,229103,323465,226303,543181,552568
4,270758,170901,263036,163641,434399,433937
5,255813,189578,249971,181020,436833,439549
...,...,...,...,...,...,...
96,164124,295383,155973,257820,421944,451356
97,163247,290577,155039,265314,428561,445616
98,198313,290779,193174,289038,487351,483953


In [13]:
df['all'] = df['F1+R2'] + df['F2+R1']

In [14]:
df

F1  R1  F2  R2  F1+R2  F2+R1  all
junction_id            offset                                   
10_100008748_100010822 1        1   0   0   0      1      0    1
                       2        1   0   0   0      1      0    1
                       5        1   0   0   0      1      0    1
                       7        2   0   0   1      3      0    3
                       8        0   0   1   0      0      1    1
...                            ..  ..  ..  ..    ...    ...  ...
Y_9573408_9578068      29       1   0   0   0      1      0    1
Y_9582135_9582487      40       1   0   0   0      1      0    1
Y_9582573_9589958      45       0   0   0   1      1      0    1
Y_9590955_9593985      35       0   0   0   1      1      0    1
                       82       1   0   0   0      1      0    1

[6502693 rows x 7 columns]

In [15]:
import numpy as np

In [17]:
df.\
drop(['F1', 'R1', 'F2', 'R2', 'F1+R2', 'F2+R1'], axis=1).\
groupby('junction_id').\
apply(lambda x: np.log2(x.sum()) - (x * np.log2(x)).sum() / x.sum())

,all
junction_id,
10_100008748_100010822,5.184441
10_100010933_100011323,4.424318
10_100010954_100011323,0.000000
10_100011459_100012110,4.569773
10_100011459_100013310,1.584963
...,...
hs37d5_94447_94548,1.000000
hs37d5_9485627_9485684,1.000000
hs37d5_9860395_9886374,3.378783


In [36]:
df.\
drop(['F1', 'R1', 'F2', 'R2', 'F1+R2', 'F2+R1'], axis=1).\
groupby('junction_id').aggregate(
    ['sum', 'count']
)

count      
                         sum count
junction_id                       
10_100008748_100010822    70    45
10_100010933_100011323    45    28
10_100010954_100011323     1     1
10_100011459_100012110    46    28
10_100011459_100013310     3     3
...                      ...   ...
hs37d5_94447_94548         2     2
hs37d5_9485627_9485684     2     2
hs37d5_9860395_9886374    14    11
hs37d5_9886442_9899937     7     6
hs37d5_9972460_9975092     1     1

[357000 rows x 2 columns]

In [3]:
pcawg = pd.read_table('sorted_output/python_PCAWG.sorted.tsv', header=None)

In [11]:
pcawg.columns = ['junction_id', 'offset', 'F1', 'R1', 'F2', 'R2']
pcawg['F1+R2'] = pcawg['F1'] + pcawg['R2']
pcawg['F2+R1'] = pcawg['F2'] + pcawg['R1']
pcawg.iloc[5:20]

,junction_id,offset,F1,R1,F2,R2,F1+R2,F2+R1
5,10_100008748_100010822,9,2,0,0,0,2,0
6,10_100008748_100010822,13,1,0,0,0,1,0
7,10_100008748_100010822,15,2,0,2,2,4,2
8,10_100008748_100010822,16,1,0,0,0,1,0
9,10_100008748_100010822,21,0,1,0,0,0,1
10,10_100008748_100010822,22,0,1,0,0,0,1
11,10_100008748_100010822,23,0,0,0,1,1,0
12,10_100008748_100010822,24,0,0,1,0,0,1
13,10_100008748_100010822,25,0,0,0,1,1,0
14,10_100008748_100010822,26,1,0,0,0,1,0


In [9]:
encode = pd.read_table('sorted_output/python_ENCODE.sorted.tsv', header=None)

In [12]:
encode.columns = ['junction_id', 'offset', 'F1', 'R1', 'F2', 'R2']
encode['F1+R2'] = encode['F1'] + encode['R2']
encode['F2+R1'] = encode['F2'] + encode['R1']
encode.iloc[5:20]

,junction_id,offset,F1,R1,F2,R2,F1+R2,F2+R1
5,chr10_100144824_100146958,17,8,0,0,0,8,0
6,chr10_100144824_100146958,18,2,0,0,0,2,0
7,chr10_100144824_100146958,28,2,0,0,0,2,0
8,chr10_100144824_100146958,33,4,0,0,0,4,0
9,chr10_100144824_100146958,45,1,0,0,0,1,0
10,chr10_100144824_100146958,51,1,0,0,0,1,0
11,chr10_100144824_100146958,53,0,0,0,1,1,0
12,chr10_100144824_100146958,58,2,0,0,0,2,0
13,chr10_100144824_100146958,69,2,0,0,0,2,0
14,chr10_100144824_100146958,70,1,0,0,0,1,0
